<a href="https://colab.research.google.com/github/annie-lyu/word-embedding/blob/master/ldaseq_abcTitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from gensim import corpora
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [14]:
import glob
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
## preparation for text preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


["haven't",
 'these',
 'of',
 'doesn',
 'being',
 'for',
 'my',
 'only',
 'below',
 'yourself',
 'they',
 'once',
 'ain',
 "needn't",
 'during',
 'hadn',
 'we',
 'nor',
 'through',
 'will',
 'don',
 'too',
 'hasn',
 "you'd",
 'a',
 'between',
 'to',
 'am',
 'if',
 "you'll",
 'until',
 'any',
 'with',
 'each',
 'aren',
 "she's",
 'has',
 'wasn',
 'how',
 'theirs',
 'shan',
 'couldn',
 "couldn't",
 'having',
 'them',
 'by',
 'their',
 'this',
 "doesn't",
 'no',
 'myself',
 'because',
 'into',
 'does',
 're',
 'herself',
 'there',
 've',
 'or',
 'she',
 'shouldn',
 'where',
 'in',
 'ma',
 'what',
 'yourselves',
 'further',
 "wouldn't",
 'isn',
 'been',
 'did',
 'needn',
 'then',
 't',
 'mustn',
 "mightn't",
 'those',
 "won't",
 'wouldn',
 'the',
 'not',
 'mightn',
 'himself',
 "mustn't",
 'whom',
 'doing',
 'such',
 'after',
 'why',
 'her',
 'over',
 'he',
 "aren't",
 'on',
 'off',
 "shan't",
 'd',
 "hadn't",
 "it's",
 'before',
 'at',
 'should',
 "shouldn't",
 'above',
 'from',
 'more',


In [16]:
import re
def removePunctuation(text):
    punctuation = '!,;:?"\'.-'
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip().lower()
 

In [17]:
## establish dictionary and remove stop words
dictionary_6k= corpora.Dictionary(line.lower().split() for line in open("/content/drive/My Drive/words/titles_6k.txt",'r', errors='ignore'))
stop_ids = [dictionary_6k.token2id[stopword] for stopword in stop_words if stopword in dictionary_6k.token2id]
dictionary_6k.filter_tokens(stop_ids)
dictionary_6k.compactify()
dictionary_6k.save("/content/drive/My Drive/words/titles_6k_dictionary")


## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/titles_6k.txt"):
            yield dictionary_6k.doc2bow(line.lower().split())

corpus_memory_friendly_6k = MyCorpus()
corpus_6k = [vector for vector in corpus_memory_friendly_6k]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/titles_6k', corpus_6k)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
dictionary_12k= corpora.Dictionary(line.lower().split() for line in open("/content/drive/My Drive/words/titles_12k.txt",'r', errors='ignore'))
stop_ids = [dictionary_12k.token2id[stopword] for stopword in stop_words if stopword in dictionary_12k.token2id]
dictionary_12k.filter_tokens(stop_ids)
dictionary_12k.compactify()
dictionary_12k.save("/content/drive/My Drive/words/titles_12k_dictionary")


## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/titles_12k.txt"):
            yield dictionary_12k.doc2bow(line.lower().split())

corpus_memory_friendly_12k = MyCorpus()
corpus_12k = [vector for vector in corpus_memory_friendly_12k]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/titles_12k', corpus_12k)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
## loading and setting
try:
  dictionary_6k = Dictionary.load("/content/drive/My Drive/words/titles_6k_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")
corpus_6k = bleicorpus.BleiCorpus('/content/drive/My Drive/words/titles_6k')

try:
  dictionary_12k = Dictionary.load("/content/drive/My Drive/words/titles_12k_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")
corpus_12k = bleicorpus.BleiCorpus('/content/drive/My Drive/words/titles_12k')


time_slice_6k = [6000]
time_slice_12k = [12000]
time_slice_12dtm = [5956, 6044]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [20]:
ldaseq_6k_5 = ldaseqmodel.LdaSeqModel(corpus=corpus_6k, id2word=dictionary_6k, time_slice=time_slice_6k, num_topics=5)
ldaseq_6k_10 = ldaseqmodel.LdaSeqModel(corpus=corpus_6k, id2word=dictionary_6k, time_slice=time_slice_6k, num_topics=10)
ldaseq_12k_10 = ldaseqmodel.LdaSeqModel(corpus=corpus_12k, id2word=dictionary_12k, time_slice=time_slice_12k, num_topics=10)
ldaseq_12k_dtm = ldaseqmodel.LdaSeqModel(corpus=corpus_12k, id2word=dictionary_12k, time_slice=time_slice_12dtm, num_topics=10)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/models/ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function i

In [22]:
#Printing Topics
ldaseq_6k_5.print_topics(time=0)

[[('news', 0.0059985081692858575),
  ('rural', 0.0047779201155514865),
  ('china', 0.004433626679386865),
  ('business', 0.0043948858863933965),
  ('labor', 0.004251118760508545),
  ('water', 0.004115738737905475),
  ('new', 0.0034023659894761747),
  ('national', 0.0032970131207797513),
  ('still', 0.003275978094936631),
  ('qld', 0.0032635970032406742),
  ('abc', 0.003252622405175814),
  ('australian', 0.0030844727925650743),
  ('tasmanian', 0.0030693143573070655),
  ('open', 0.0029836012963305888),
  ('named', 0.0029165161333855254),
  ('plan', 0.0028549053891835205),
  ('market', 0.0028155960069148464),
  ('coast', 0.0027581998292634836),
  ('markets', 0.002710601813064861),
  ('may', 0.0026658313938843843)],
 [('council', 0.00854344977926679),
  ('new', 0.00705272525107389),
  ('police', 0.0060343481628943665),
  ('nsw', 0.005408940908207149),
  ('call', 0.0048387513187494995),
  ('back', 0.004510654252652414),
  ('calls', 0.00449567475699092),
  ('ban', 0.0044913974234087805),
  (

In [23]:
ldaseq_6k_10.print_topics(time=0)

[[('fire', 0.014524675161517692),
  ('water', 0.010286020597798394),
  ('weather', 0.00904734090537707),
  ('abc', 0.008031059598670443),
  ('charged', 0.007884990624456805),
  ('iraq', 0.007525129067716312),
  ('us', 0.006841280105912653),
  ('police', 0.006460777527460369),
  ('new', 0.006459086952299379),
  ('oil', 0.005570737134252545),
  ('work', 0.005561438355967708),
  ('man', 0.005493355707866688),
  ('drug', 0.005406005062266096),
  ('hits', 0.00520408742548684),
  ('coal', 0.0051337544162880925),
  ('brisbane', 0.005123862241780327),
  ('women', 0.005086628621432053),
  ('hill', 0.005046384702406257),
  ('two', 0.005027263241897641),
  ('govt', 0.004694229226733299)],
 [('police', 0.015044289477656245),
  ('court', 0.013652171952578687),
  ('new', 0.009108564951487114),
  ('national', 0.008576662211422864),
  ('industry', 0.008298451988232364),
  ('government', 0.007324564944202472),
  ('sydney', 0.006551279557314325),
  ('accused', 0.006480484561152185),
  ('country', 0.0062

In [24]:
ldaseq_12k_10.print_topics(time=0)

[[('change', 0.007340392613423584),
  ('climate', 0.007079758113461179),
  ('election', 0.006784350389048845),
  ('police', 0.0067101953065625),
  ('urges', 0.006432142085563655),
  ('care', 0.006304139330182992),
  ('weather', 0.00610743897345364),
  ('still', 0.006026413987281553),
  ('rate', 0.005917291592061095),
  ('report', 0.0058388263408629265),
  ('tasmania', 0.005775079941847599),
  ('fires', 0.005769988563746352),
  ('us', 0.005755730814825704),
  ('deal', 0.005705936060161448),
  ('plant', 0.005605597624361099),
  ('warning', 0.0054783136249092505),
  ('council', 0.005472037367012381),
  ('beach', 0.005447830067355544),
  ('charge', 0.005386987536489336),
  ('makes', 0.005273056895239839)],
 [('man', 0.0252424095569462),
  ('police', 0.022390643663027513),
  ('charged', 0.013089669436938817),
  ('sydney', 0.01296999348500899),
  ('found', 0.009927110664908083),
  ('missing', 0.009788413656374144),
  ('two', 0.008851691075036245),
  ('dead', 0.00854120654203568),
  ('abc', 0

In [29]:
ldaseq_12k_dtm.print_topics(time=0)


[[('south', 0.011612643319170718),
  ('us', 0.010680599460112525),
  ('support', 0.008730015685130942),
  ('abc', 0.008333506158781817),
  ('business', 0.007699147263104096),
  ('talks', 0.007057673235808379),
  ('new', 0.006843402773319586),
  ('act', 0.006613401924111398),
  ('island', 0.006309672233315538),
  ('housing', 0.006195258916562738),
  ('govt', 0.006018535793072738),
  ('centre', 0.0059789593024909765),
  ('home', 0.005613834018129105),
  ('iraq', 0.005558529418374949),
  ('group', 0.005424235650697051),
  ('pm', 0.00540647169632428),
  ('report', 0.005385852634494017),
  ('christmas', 0.00502161260103651),
  ('china', 0.004877458736380696),
  ('mayor', 0.004862655600936611)],
 [('police', 0.03206513913898626),
  ('charged', 0.01679766161099053),
  ('man', 0.015643932136491318),
  ('woman', 0.015564454619658281),
  ('rural', 0.010108318504890591),
  ('nsw', 0.009281560418768297),
  ('attack', 0.008833966624574507),
  ('missing', 0.008681587894921251),
  ('assault', 0.00825

In [31]:
ldaseq_12k_dtm.print_topics(time=1)

[[('south', 0.011704300823461793),
  ('us', 0.01069405088150145),
  ('support', 0.008743843945396703),
  ('abc', 0.008583045931368109),
  ('business', 0.007747854117360487),
  ('talks', 0.007065111608448174),
  ('new', 0.006857608287951159),
  ('act', 0.006632723220647422),
  ('island', 0.006399716081058178),
  ('housing', 0.006205051526261382),
  ('centre', 0.005979726337088804),
  ('govt', 0.005897601979518541),
  ('home', 0.0056185515250799754),
  ('iraq', 0.005526721255318767),
  ('group', 0.00543085697890832),
  ('pm', 0.005403073132199497),
  ('report', 0.005396289861397773),
  ('christmas', 0.005105159947628073),
  ('china', 0.004893579262825557),
  ('mayor', 0.004772252302240911)],
 [('police', 0.028584623911807844),
  ('charged', 0.016931475206717922),
  ('man', 0.01575226697709117),
  ('woman', 0.015683072717334025),
  ('rural', 0.010307328084060425),
  ('nsw', 0.009594949174419118),
  ('attack', 0.008867182118969995),
  ('missing', 0.008728917518021632),
  ('assault', 0.0082

In [30]:
#Topic Evolution
ldaseq_12k_dtm.print_topic_times(topic=0)

[[('south', 0.011612643319170718),
  ('us', 0.010680599460112525),
  ('support', 0.008730015685130942),
  ('abc', 0.008333506158781817),
  ('business', 0.007699147263104096),
  ('talks', 0.007057673235808379),
  ('new', 0.006843402773319586),
  ('act', 0.006613401924111398),
  ('island', 0.006309672233315538),
  ('housing', 0.006195258916562738),
  ('govt', 0.006018535793072738),
  ('centre', 0.0059789593024909765),
  ('home', 0.005613834018129105),
  ('iraq', 0.005558529418374949),
  ('group', 0.005424235650697051),
  ('pm', 0.00540647169632428),
  ('report', 0.005385852634494017),
  ('christmas', 0.00502161260103651),
  ('china', 0.004877458736380696),
  ('mayor', 0.004862655600936611)],
 [('south', 0.011704300823461793),
  ('us', 0.01069405088150145),
  ('support', 0.008743843945396703),
  ('abc', 0.008583045931368109),
  ('business', 0.007747854117360487),
  ('talks', 0.007065111608448174),
  ('new', 0.006857608287951159),
  ('act', 0.006632723220647422),
  ('island', 0.00639971608

In [27]:
## word distribution in a specific document
#words = [dictionary[word_id] for word_id, count in corpus[558]]
#print (words)

NameError: ignored

In [ ]:
## A specific document's latent topic distribution
#doc = ldaseq.doc_topics(558)
#print(doc)

In [28]:
## changing rate of topic factors(words in topics)
ldaseq_12k_dtm.print_topic_times(1)

##Note: the chain_variance parameter could affect the result

[[('police', 0.03206513913898626),
  ('charged', 0.01679766161099053),
  ('man', 0.015643932136491318),
  ('woman', 0.015564454619658281),
  ('rural', 0.010108318504890591),
  ('nsw', 0.009281560418768297),
  ('attack', 0.008833966624574507),
  ('missing', 0.008681587894921251),
  ('assault', 0.008256886212217701),
  ('arrested', 0.006314635623597327),
  ('sa', 0.006189396178841792),
  ('school', 0.005439244379665249),
  ('fatal', 0.005317904891553859),
  ('second', 0.005101377415257776),
  ('search', 0.005081340275325346),
  ('sydney', 0.005028887389537389),
  ('murder', 0.004929625813795862),
  ('driver', 0.004924549399250471),
  ('bank', 0.00483480855187827),
  ('hunt', 0.004748240697176902)],
 [('police', 0.028584623911807844),
  ('charged', 0.016931475206717922),
  ('man', 0.01575226697709117),
  ('woman', 0.015683072717334025),
  ('rural', 0.010307328084060425),
  ('nsw', 0.009594949174419118),
  ('attack', 0.008867182118969995),
  ('missing', 0.008728917518021632),
  ('assault',